# Tutoriel RAG : Comment charger les jeux de données de MediaTech depuis Hugging Face et les utiliser dans un système RAG ?

Ce notebook montre comment construire un pipeline de **Retrieval-Augmented Generation (RAG)** en utilisant :
- **Hugging Face Datasets** : charger des documents juridiques pré-calculés (embeddings) depuis le dataset LEGI
- **Qdrant** : base de données vectorielle pour une recherche de similarité efficace (vecteurs denses + vecteurs sparse)
- **API compatible OpenAI** : ici nous utilisons OpenGateLLM, l'API LLM du gouvernement français compatible OpenAI pour les embeddings et l'inférence

## Prérequis
- Une instance Qdrant en fonctionnement (par défaut : `localhost:6333`)
- Une clé API pour l'API compatible OpenAI (ici https://albert.api.etalab.gouv.fr)
- Packages requis : `fastembed`, `qdrant-client`, `openai`, `datasets`, `pandas`

---

## 1. Configuration

In [ ]:
# Installer les packages requis
#%pip install pandas fastembed qdrant-client openai datasets

In [ ]:
from fastembed import SparseTextEmbedding
from qdrant_client import QdrantClient

# === Configuration de l'API ===
# Nous utilisons pour cet exemple OpenGateLLM
# une API compatible OpenAI du gouvernement français pour les LLMs et les embeddings
API_KEY = "changeme"  # Remplacez par votre clé API
API_URL = "https://albert.api.etalab.gouv.fr/v1" 

# === Configuration de la base de données vectorielle ===
# Connexion à l'instance Qdrant locale pour stocker et rechercher les vecteurs
client = QdrantClient(url="http://localhost", port=6333)

# === Modèle d'embedding sparse ===
# Modèle BM25 pour les embeddings sparse basés sur les mots-clés (utilisé dans la recherche hybride)
bm25_embedding_model = SparseTextEmbedding("Qdrant/bm25")

---

## 2. Fonctions principales

Cette section définit les fonctions principales pour :
1. **Génération d'embeddings** : Convertir le texte en vecteurs denses avec BGE-M3
2. **Récupération** : Rechercher dans la base de données vectorielle en utilisant la recherche hybride (dense + sparse)
3. **Inférence** : Générer des réponses avec un LLM en streaming
4. **Construction du prompt** : Construire des prompts RAG avec le contexte récupéré

### 2.1 Génération d'embeddings

Génère des embeddings denses en utilisant le modèle BGE-M3 via OpenGateLLM. Ce modèle est multilingue et fonctionne bien pour les textes juridiques français.

In [ ]:
from openai import OpenAI


def generate_embeddings(
    data: str | list[str], model: str = "BAAI/bge-m3"
) -> list[float]:
    """
    Génère des embeddings pour un texte donné en utilisant un modèle spécifié.

    Args:
        data (str ou list[str]) : L'entrée pour laquelle générer des embeddings.
        model (str, optionnel) : L'identifiant du modèle à utiliser pour générer les embeddings. Par défaut "BAAI/bge-m3".

    Returns:
        list[float] : Le vecteur d'embedding pour le texte d'entrée.

    Raises:
        Toute exception levée par le client OpenAI pendant le processus de génération d'embeddings.

    Note:
        Nécessite une configuration correcte de API_URL et API_KEY pour le client OpenAI.
    """
    client_openai = OpenAI(base_url=API_URL, api_key=API_KEY)
    vector = client_openai.embeddings.create(
        input=data, model=model, encoding_format="float"
    )
    embeddings = [item.embedding for item in vector.data]

    return embeddings

### 2.2 Récupération hybride

Récupère les documents pertinents en utilisant la **Fusion de Rangs Réciproques (RRF)** pour combiner :
- **Recherche dense** : Similarité sémantique utilisant les embeddings BGE-M3
- **Recherche sparse** : Correspondance de mots-clés utilisant BM25

Cette approche hybride améliore la qualité de la récupération en exploitant à la fois la compréhension sémantique et la correspondance exacte de mots-clés.

In [ ]:
def inference(
    chat_messages: list[dict],
    model: str = "mistralai/Mistral-Small-3.2-24B-Instruct-2506",  # Changez selon votre modèle préféré
    return_output: bool = False,
    print_inference: bool = True,
    print_prompt: bool = False,
    max_tokens: int = 2000,
):
    """
    Effectue une inférence en utilisant un modèle de chat avec sortie en streaming.
    Args:
        chat_messages (list[dict]) : Les messages de chat à envoyer au modèle.
        model (str, optionnel) : Le nom du modèle à utiliser pour l'inférence. Par défaut "mistralai/Mistral-Small-3.2-24B-Instruct-2506".
        return_output (bool, optionnel) : Si True, retourne la sortie complète sous forme de chaîne. Par défaut False.
        print_inference (bool, optionnel) : Si True, affiche la sortie de l'inférence en temps réel. Par défaut True.
        print_prompt (bool, optionnel) : Si True, affiche les messages du prompt. Par défaut False.
        max_tokens (int, optionnel) : Le nombre maximum de tokens à générer. Par défaut 2000.
    """
    client = OpenAI(
        api_key=API_KEY,
        base_url=API_URL,
    )

    if print_prompt:
        print(chat_messages)

    # streaming de chat.completions
    chat_response = client.chat.completions.create(
        model=model,  # doit être le nom du modèle déployé sur le serveur API
        stream=True,
        # top_p=0.9,
        temperature=0.1,
        max_tokens=max_tokens,
        messages=chat_messages,
    )
    output = ""
    for chunk in chat_response:
        try:
            if chunk.choices[0].delta.content:
                output += chunk.choices[0].delta.content
                if print_inference:
                    print(chunk.choices[0].delta.content, flush=True, end="")
        except Exception as e:
            continue

    if return_output:
        return output


# Exemple d'utilisation
print(inference(chat_messages=[{"role": "user", "content": "Salut ca va ?"}]))

### 2.3 Inférence LLM

Diffuse les réponses du modèle Mistral via l'API Albert. Le streaming offre une meilleure expérience utilisateur en affichant les tokens au fur et à mesure de leur génération.

In [ ]:
from qdrant_client import models


def retrieval(
    query: str,
    collection_name="legi_code_travail",
    hybrid_search: bool = True,
    limit: int = 10,
):
    """
    Récupère les documents pertinents d'une collection Qdrant basée sur une requête.
    Args:
        query (str) : La requête de recherche.
        collection_name (str, optionnel) : Le nom de la collection Qdrant à rechercher. Par défaut "legi_code_travail".
        hybrid_search (bool, optionnel) : Si True, utilise la recherche hybride (embedding + sparse). Par défaut True.
        limit (int, optionnel) : Le nombre maximum de résultats à retourner. Par défaut 10.
    """
    embedding = generate_embeddings(query)[0]
    sparse_query_vector = next(bm25_embedding_model.query_embed(query))

    if hybrid_search:
        # Effectuer la recherche
        search_results = client.query_points(
            collection_name=collection_name,
            prefetch=[
                models.Prefetch(
                    query=embedding,
                    using="BAAI/bge-m3",
                    limit=2*limit,
                ),
                models.Prefetch(
                    query=models.SparseVector(**sparse_query_vector.as_object()),
                    using="bm25",
                    limit=2*limit,
                ),
            ],
            with_payload=True,
            query=models.FusionQuery(fusion=models.Fusion.RRF),
            limit=limit,
        )
    else:
        # Effectuer la recherche
        search_results = client.query_points(
            collection_name=collection_name,
            query=embedding,
            using="BAAI/bge-m3",
            limit=limit,
            with_payload=True,
        )

    # Afficher le résultat le plus proche
    results = []
    if search_results:
        for result in search_results.points:
            results.append({"payload": result.payload, "score": result.score})
            # print("Payload du point le plus proche:", result)
        return results
    else:
        print("Aucun résultat trouvé")

### 2.4 Construction du prompt RAG

Construit un prompt qui inclut les documents récupérés comme contexte. Le LLM utilisera ces documents pour générer une réponse informée.

In [ ]:
def make_prompt(
    query: str,
    system_prompt: str = "Tu es un assistant IA utile qui répond aux questions des utilisateurs en utilisant des documents pertinents fournis.",
    hybrid_search: bool = True,
    collection_name: str = "legi_code_travail",
    limit: int = 5,
):
    chunks = []
    results = retrieval(
        query=query, collection_name=collection_name, hybrid_search=hybrid_search
    )
    chunks.extend(results[k].get("payload") for k in range(len(results)))

    top_chunks = chunks[:limit]
    chat_messages = [{"role": "system", "content": system_prompt}]

    prompt = f"""
    Voici ci dessous les documents pertinents pour répondre à la question suivante : {query}\n
    """
    for chunk in top_chunks:
        prompt += f"""
        <<< {chunk.get("chunk_text", "")} >>>
        """

    chat_messages.append({"role": "user", "content": prompt})

    return chat_messages, top_chunks

### 2.5 Fonctions utilitaires

Fonctions utilitaires, par exemple pour générer des UUIDs déterministes à partir des IDs de chunks pour l'identification des points Qdrant.

In [ ]:
import hashlib
import uuid


def string_to_uuid(s: str) -> str:
    hash_bytes = hashlib.sha256(str(s).encode()).digest()[:16]
    return str(uuid.UUID(bytes=hash_bytes))

---

## 3. Chargement du dataset et création de la base de données vectorielle

Nous chargeons le **Code du Travail** français depuis le dataset [AgentPublic/legi](https://huggingface.co/datasets/AgentPublic/legi) sur Hugging Face. Ce dataset contient :
- Des articles juridiques pré-découpés en chunks
- Des embeddings BGE-M3 pré-calculés
- Des métadonnées (statut, ID d'article, etc.)

Nous filtrons pour ne garder que les articles actuellement en vigueur (`VIGUEUR`) ou qui seront abrogés dans le futur (`ABROGE_DIFF`).

In [ ]:
import json

import pandas as pd
from datasets import load_dataset

# Charger le sous-ensemble Code du Travail depuis le dataset LEGI
# Le dataset est disponible sur : https://huggingface.co/datasets/AgentPublic/legi
dataset = load_dataset(
    "AgentPublic/legi", data_files="data/legi-latest/legi_code_du_travail/*.parquet"
)

df = pd.DataFrame(dataset["train"])
print(f"Total d'articles chargés : {len(df)}")

# Filtrer pour ne garder que les articles valides :
# - VIGUEUR : Actuellement en vigueur
# - ABROGE_DIFF : Sera abrogé à une date future (encore valide maintenant)
df = df[df["status"].isin(["VIGUEUR", "ABROGE_DIFF"])]
print(f"Articles après filtrage : {len(df)}")

# Parser les embeddings pré-calculés depuis les chaînes JSON vers des listes
df["embeddings_bge-m3"] = df["embeddings_bge-m3"].apply(json.loads)

# Aperçu de la structure du dataset
df.head()

### Créer une collection Qdrant avec des vecteurs hybrides

Nous créons une collection Qdrant avec deux types de vecteurs :
1. **Vecteurs denses** (`BAAI/bge-m3`) : Embeddings sémantiques pré-calculés depuis le dataset
2. **Vecteurs sparse** (`bm25`) : Calculés à la volée en utilisant le modèle BM25 avec pondération IDF

In [ ]:
from qdrant_client import models
from qdrant_client.models import PointStruct
from tqdm import tqdm

collection_name = "legi_code_travail"
embedding_dim = len(df["embeddings_bge-m3"].iloc[0])

# Créer la collection si elle n'existe pas
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            # Configuration des vecteurs denses pour la recherche sémantique
            "BAAI/bge-m3": models.VectorParams(
                size=embedding_dim, distance=models.Distance.COSINE
            )
        },
        sparse_vectors_config={
            # Configuration des vecteurs sparse pour la recherche BM25 par mots-clés
            "bm25": models.SparseVectorParams(modifier=models.Modifier.IDF)
        },
    )
    print(f"Nouvelle collection créée : {collection_name}")
else:
    print(f"La collection '{collection_name}' existe déjà")

# Préparer les points avec les vecteurs denses et sparse
print("Préparation des points avec les embeddings...")
points = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Calcul des embeddings BM25"):
    # Calculer les embeddings sparse BM25 pour la recherche hybride
    bm25_embeddings = list(bm25_embedding_model.passage_embed(row["chunk_text"]))
    
    points.append(
        PointStruct(
            id=string_to_uuid(row["chunk_id"]),
            vector={
                "BAAI/bge-m3": row["embeddings_bge-m3"],  # Vecteur dense
                "bm25": bm25_embeddings[0].as_object(),   # Vecteur sparse
            },
            payload={
                "chunk_text": row["chunk_text"],
                # Inclure toutes les colonnes de métadonnées sauf les embeddings
                **{
                    col: row[col]
                    for col in df.columns
                    if col not in ["embeddings_bge-m3", "chunk_text"]
                },
            },
        )
    )

# Insérer les points par lots pour plus d'efficacité
batch_size = 100
print("Insertion des points dans Qdrant...")
for i in tqdm(range(0, len(points), batch_size), desc="Téléchargement des lots"):
    client.upsert(collection_name=collection_name, points=points[i : i + batch_size])

print(f"\nCollection '{collection_name}' prête avec {len(points)} vecteurs (dimension : {embedding_dim})")

---

## 4. Test du pipeline RAG

Testons maintenant notre système RAG avec une question sur le droit du travail français. Le pipeline va :
1. **Récupérer** les articles juridiques pertinents en utilisant la recherche hybride
2. **Augmenter** le prompt avec le contexte récupéré
3. **Générer** une réponse informée en utilisant le LLM

In [ ]:
# Définir le prompt système pour l'assistant juridique
system_prompt = """Tu es un assistant IA utile et expert dans le domaine juridique qui répond aux questions des utilisateurs en utilisant des documents pertinents fournis.
Si tu ne sais pas, réponds que tu ne sais pas. 
"""

# Poser une question sur le droit du travail français
question = "Quelle est la durée journalière légale du travail en France ?"

# Construire le prompt RAG avec le contexte récupéré
chat_messages, top_chunks = make_prompt(
    query=question,
    system_prompt=system_prompt,
    collection_name="legi_code_travail",
    hybrid_search=True,  # Utiliser la recherche dense et sparse
    limit=7,             # Récupérer les 7 meilleurs documents
)

# Afficher les documents récupérés (optionnel - décommenter pour voir les sources)
# print(f"{len(top_chunks)} documents pertinents récupérés\n")
# for k, chunk in enumerate(top_chunks):
#     print(f"---- Document {k+1} ----")
#     print(chunk.get("chunk_text")[:300] + "..." if len(chunk.get("chunk_text", "")) > 300 else chunk.get("chunk_text"))
#     print()

# Générer la réponse en utilisant le LLM
print("=" * 50)
print("Réponse :")
print("=" * 50)
inference(
    chat_messages=chat_messages,
    model="mistralai/Mistral-Small-3.2-24B-Instruct-2506",
    return_output=False,
    print_inference=True,
    print_prompt=False,
    max_tokens=2000,
)